# Import Libraries

In [ ]:
import numpy as np
import pandas as pd

# Read webscraped recipe topic data

In [ ]:
culturedf = pd.read_csv('full_culture_tastyscrape.csv')
dietdf = pd.read_csv('full_diet_tastyscrape.csv')
mealdf = pd.read_csv('full_meal_tastyscrape.csv')

### Drop duplicates by URL

In [ ]:
dietdf.drop_duplicates(subset=['URL'], inplace=True)

In [ ]:
culturedf.drop_duplicates(subset=['URL'], inplace=True)

In [ ]:
mealdf.drop_duplicates(subset=['URL'], inplace=True)

### Join topic data into one dataframe by URL

In [ ]:
topicdf = pd.merge(mealdf, dietdf, on='URL', how='outer')
topicdf = pd.merge(topicdf, culturedf, on='URL', how='outer')

In [ ]:
topicdf['URL'].duplicated().sum()

In [ ]:
urls = topicdf[['URL', 'meal', 'diet', 'culture']]

In [ ]:
urls

In [ ]:
urls.to_csv('recipeurls.csv')

# Drop duplicates of recipes data

In [ ]:
df = pd.read_csv('tastyrecipes.csv')

In [ ]:
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)

In [ ]:
df.to_csv('tastyrecipes.csv')

# Drop duplicates of ingredients data

In [ ]:
ingdf = pd.read_csv('tastyingredients.csv')

In [ ]:
ingdf[ingdf.duplicated()]

In [ ]:
len(ingdf['url'].unique())

In [ ]:
uniqueurls = list(ingdf['url'].unique())

In [ ]:
# there are multiple rows with the same url, but have different ingredients (one recipe webpage visited with multiple ingredients)
# we only want to remove the duplicates that are repeated entries at different instances when webscraping
# ie. remove duplicates by recipe webpages that were revisited

urlsused = []
dups = []
a = ''
for i in range(1, ingdf.shape[0]):
    if ingdf['url'].iloc[i] not in urlsused:
        urlsused.append(ingdf['url'][i])
    elif ingdf['url'][i] != ingdf['url'][i-1]:
        a = ingdf['url'][i]
        dups.append(i)
    elif a == ingdf['url'][i]:
        dups.append(i)
    else:
        pass
        

In [ ]:
len(urlsused)

In [ ]:
len(dups)

In [ ]:
cleanings = ingdf.drop(ingdf.index[dups])

In [ ]:
cleanings.reset_index(drop=True, inplace=True)

### Check that all duplicates are removed

In [ ]:
urlsused = []
dups = []
a = ''
for i in range(1, cleanings.shape[0]):
    if cleanings['url'].iloc[i] not in urlsused:
        urlsused.append(cleanings['url'][i])
    elif cleanings['url'][i] != cleanings['url'][i-1]:
        a = cleanings['url'][i]
        dups.append(i)
    elif a == cleanings['url'][i]:
        dups.append(i)
    else:
        pass

In [ ]:
len(urlsused)

In [ ]:
len(dups)

In [ ]:
cleanings

In [ ]:
cleanings.to_csv('tastyingredients.csv')

# Prepare final dataset for SQL

The datasets, recipe topics, ingredients and recipes are connected by the URL.
Unique URLs that are not found in all 3 csv files must be removed

In [ ]:
ingred = pd.read_csv('tastyingredients.csv')
recipe = pd.read_csv('tastyrecipes.csv')
topic = pd.read_csv('tastytopics.csv')

In [ ]:
recipe.drop_duplicates(subset=['url'], inplace=True)

In [ ]:
recipe['url'].duplicated().sum()

In [ ]:
recipe.dropna(inplace=True)

In [ ]:
recipe.isnull().sum()

In [ ]:
recipe.to_csv('tastyrecipes.csv')

In [ ]:
dropurls = []
for a in ingred['url']:
    if a in list(recipe['url']):
        pass
    else:
        dropurls.append(a)

In [ ]:
drop = set(dropurls)

In [ ]:
drop

In [ ]:
for i in drop:
    recipe = recipe[recipe['url'] != i]

In [ ]:
for i in drop:
    ingred = ingred[ingred['url'] != i]

In [ ]:
ingred

In [ ]:
ingred.to_csv('tastyingredients.csv')

In [ ]:
dropurls = []
for a in topic['URL']:
    if a in list(recipe['url']):
        pass
    else:
        dropurls.append(a)

In [ ]:
dropurls

In [ ]:
for i in drop:
    topic = topic[topic['URL'] != i]

In [ ]:
topic.to_csv('tastytopics.csv')